# Computer Vision – Content Based Image Retrieval

Los motores de búsqueda de imágenes abarcan métodos para hacer que un conjunto de datos de imágenes pueda buscarse visualmente utilizando solo el contenido de la imagen. Los académicos lo llaman Recuperación de imágenes basadas en contenido (CBIR). Los motores de búsqueda de imágenes dependen de la extracción de características de las imágenes y, a continuación, comparan las similitudes en función de los vectores de características extraídos y la métrica de la distancia.

En general, suele haber tres tipos de motores de búsqueda de imágenes: búsqueda por metadatos, búsqueda por ejemplo y un enfoque híbrido de los dos.

- **Definición del descriptor de la imagen:** en esta fase, debemos decidir qué aspecto de la imagen desea describir. ¿Te interesa el color de la imagen? ¿La forma de un objeto en la imagen? ¿O quieres caracterizar la textura?


- **Extracción de características e indexación de su conjunto de datos:** Ahora que hemos definido nuestro descriptor de imagen, su trabajo consiste en aplicar este descriptor de imagen a cada imagen en su conjunto de datos, extraer características de estas imágenes y escribirlas en el almacenamiento (por ejemplo, archivo CSV... Redis, etc.) para que luego puedan compararse por similitud. Además, debe considerar si alguna estructura de datos especializada se utilizará para facilitar una búsqueda más rápida.


- **Definición de la métrica de similitud:** Ahora que tenemos una colección de vectores de características (quizás índice). Pero, ¿cómo vas a compararlos por similitud? Las opciones populares incluyen la distancia euclidiana, la distancia del coseno...etc, pero la elección real depende en gran medida de:

    1. El conjunto de datos
    2. Los tipos de características que vamos a extraer.


- **Búsqueda:** El paso final es realizar una búsqueda real. Un usuario enviará una imagen de consulta a su sistema (desde un formulario de carga o mediante una aplicación móvil, por ejemplo) y su trabajo consistirá en:

    1. Extraer características de esta imagen de consulta.
    2. Aplicar su función de similitud para comparar Las características de consulta a las características ya indexadas. A partir de ahí, simplemente devuelve los resultados más relevantes de acuerdo con su función de similitud. 
    
Estos son los cuatro pasos más básicos de cualquier sistema CBIR. A medida que se vuelven más complejos y utilizan diferentes representaciones de características, la cantidad de pasos aumenta y agregará una cantidad significativa de subpasos a cada paso mencionado anteriormente.


### Setup

In [4]:
from cbir.rgbhistogram import RGBHistogram
from cbir.searcher import Searcher
import numpy as np
import pickle
import cv2
import glob
import os
from PIL import Image
import imutils
from imutils.paths import list_images

## Create Index

In [6]:
# Inicializaremos el diccionario del Index para almacenar nuestro nuestro de
# imágenes, con la 'clave' del diccionario como nombre de archivo 
# y el 'valor' nuestras características de la imagen o vector
index = {}

# Inicializar nuestro descriptor de imagen - un histograma 3D RGB con
# 8 bins o contenedores por canal
desc = RGBHistogram([8, 8, 8])

for imagePath in list_images("images"):
    # extraer nuestro ID único de la imagen (por ejemplo, el nombre del archivo)
    print(imagePath)
    path = imagePath.split(os.sep)
    k = path[1]
    print(k)
 
    # Cargamos la imagen, la describimos usando nuestro descriptor de histograma RGB,
    # y actualizamos el Index
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (400, 166))
    features = desc.describe(image)
    index[k] = features

f = open("index.pickle", "wb")
f.write(pickle.dumps(index))
f.close()
 
# show how many images we indexed
print("[INFO] done...indexed {} images".format(len(index)))

images\coruscant-001.png
coruscant-001.png
images\coruscant-002.png
coruscant-002.png
images\coruscant-003.png
coruscant-003.png
images\coruscant-004.png
coruscant-004.png
images\coruscant-005.png
coruscant-005.png
images\dagobah-001.png
dagobah-001.png
images\dagobah-002.png
dagobah-002.png
images\dagobah-003.png
dagobah-003.png
images\dagobah-004.png
dagobah-004.png
images\dagobah-005.png
dagobah-005.png
images\hoth-001.png
hoth-001.png
images\hoth-002.png
hoth-002.png
images\hoth-003.png
hoth-003.png
images\hoth-004.png
hoth-004.png
images\hoth-005.png
hoth-005.png
images\naboo-001.png
naboo-001.png
images\naboo-002.png
naboo-002.png
images\naboo-003.png
naboo-003.png
images\naboo-004.png
naboo-004.png
images\naboo-005.png
naboo-005.png
images\tatooine-001.png
tatooine-001.png
images\tatooine-002.png
tatooine-002.png
images\tatooine-003.png
tatooine-003.png
images\tatooine-004.png
tatooine-004.png
images\tatooine-005.png
tatooine-005.png
[INFO] done...indexed 25 images


## Searching

In [9]:
queryImage = cv2.imread("queries/coruscant-query.png")
cv2.imshow("Query", queryImage)

print("query: %s" % ("queries/shire-query.png"))

# Describir la querie con un histograma 3D RGB con 8 bins por canal
desc = RGBHistogram([8, 8, 8])
queryFeatures = desc.describe(queryImage)

# Cargar el índice realizar la búsqueda
index = pickle.loads(open("index.pickle", "rb").read())
searcher = Searcher(index)
results = searcher.search(queryFeatures)
 
# Inicializar los dos conjuntos de salida para mostrar nuestros resultados -
# tenemos un total de 26 imágenes en el Index, pero solo mostraremos
# los 10 mejores resultados. 5 imágenes por conjunto, con
# imágenes que son 400x166 pixeles para no ocupar mucho
montageA = np.zeros((166 * 5, 400, 3), dtype = "uint8")
montageB = np.zeros((166 * 5, 400, 3), dtype = "uint8")
 
# loop esobre los diez primeros resultados
for j in range(0, 10):
    # Extraer el resultado (estamos usando row-major order) y
    # cargar la imagen del resultado
    (score, imageName) = results[j]
    path = os.path.join("images", imageName)
    result = cv2.imread(path)
    print("\t{}. {} : {:.3f}".format(j + 1, imageName, score))
 
    # verificar si se debe usar el primer conjunto
    if j < 5:
        montageA[j * 166:(j + 1) * 166, :] = result
 
    # de lo contrario, se debe utilizar el segundo conjunto.
    else:
        montageB[(j - 5) * 166:((j - 5) + 1) * 166, :] = result
        
cv2.imshow("Results 1-5", montageA)
cv2.imshow("Results 6-10", montageB)
cv2.waitKey(0)
cv2.destroyAllWindows()

query: queries/shire-query.png
	1. coruscant-004.png : 0.699
	2. dagobah-002.png : 0.803
	3. coruscant-005.png : 1.036
	4. dagobah-001.png : 1.073
	5. dagobah-003.png : 1.258
	6. naboo-001.png : 1.401
	7. dagobah-004.png : 1.451
	8. dagobah-005.png : 1.720
	9. coruscant-003.png : 1.805
	10. tatooine-005.png : 1.829
